In [13]:
# %pip install langchain
# %pip install langchain-openai

import os
import sys
from typing import List, Dict, Any
from langchain_core.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate
from langchain_openai.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from pydantic.v1 import BaseModel, Field
from openai import OpenAIError

os.environ["OPENAI_API_KEY"] = ""

In [16]:
#from langchain_openai import ChatOpenAI
#llm = ChatOpenAI()
#llm.invoke("how can langsmith help with testing?")

In [15]:
class BusinessName(BaseModel):
    name: str = Field(description="The name of the business")
    rating_score: float = Field(description="The rating score of the business. 0 is the worst, 10 is the best.")

class BusinessNames(BaseModel):
    names: List[BusinessName] = Field(description="A list of business names")

def create_chat_model(temperature: float = 0.0) -> ChatOpenAI:
    try:
        return ChatOpenAI(temperature=temperature)
    except OpenAIError as e:
        print(f"Error creating ChatOpenAI model: {e}")
        sys.exit(1)

def create_parser() -> PydanticOutputParser:
    return PydanticOutputParser(pydantic_object=BusinessNames)

def create_chat_prompt(template: str) -> ChatPromptTemplate:
    system_message_prompt = SystemMessagePromptTemplate.from_template(template)
    return ChatPromptTemplate.from_messages([system_message_prompt])

def generate_names(model: ChatOpenAI, chat_prompt: ChatPromptTemplate, parser: PydanticOutputParser,
                   input_data: Dict[str, Any]) -> BusinessNames:
    try:
        prompt_and_model = chat_prompt | model
        result = prompt_and_model.invoke(input_data)
        return parser.parse(result.content)
    except OpenAIError as e:
        print(f"Error generating names: {e}")
        return BusinessNames(names=[])
    except ValueError as e:
        print(f"Error parsing output: {e}")
        return BusinessNames(names=[])

def main():
    if 'OPENAI_API_KEY' not in os.environ:
        print("Error: OPENAI_API_KEY environment variable is not set.")
        sys.exit(1)

    principles = """
    - The name should evoke the core values of the {industry} industry.
    - Incorporate a futuristic or innovative element in the name.
    - Ensure the name has a positive connotation in most languages.
    - The name should be between 4 to 8 characters long.
    - Avoid using common words or existing brand names.
    - The name should work well as a domain name (consider .com availability).
    """

    template = """Generate 5 business names for a new start up company in the {industry} industry.
    You must follow the following principles: {principles}
    {format_instructions}
    """

    try:
        model = create_chat_model()
        parser = create_parser()
        chat_prompt = create_chat_prompt(template)

        input_data = {
            "principles": principles,
            "industry": "Artificial Intelligence",
            "format_instructions": parser.get_format_instructions(),
        }

        # Method 1: Using generate_names function
        result = generate_names(model, chat_prompt, parser, input_data)
        print("Method 1 result:")
        print(result)
        print("\n" + "="*50 + "\n")

        # Method 2: Using chain
        chain = chat_prompt | model | parser
        result = chain.invoke(input_data)
        print("Method 2 result:")
        print(result)
        print("\n" + "="*50 + "\n")

        # Method 3: Using model.invoke() with format_prompt()
        chat_response_output = model.invoke(chat_prompt.format_prompt(**input_data).to_messages())
        pydantic_schema: BusinessNames = parser.parse(chat_response_output.content)
        print("Method 3 result:")
        print(pydantic_schema)
        print("\nJSON output:")
        print(pydantic_schema.json())

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        sys.exit(1)

if __name__ == "__main__":
    main()

Method 1 result:
names=[BusinessName(name='SynthAI', rating_score=9.2), BusinessName(name='NeuroIQ', rating_score=9.5), BusinessName(name='CogniTek', rating_score=8.8), BusinessName(name='Mindsync', rating_score=9.0), BusinessName(name='IntelliX', rating_score=9.3)]


Method 2 result:
names=[BusinessName(name='SynthAI', rating_score=9.2), BusinessName(name='NeuroIQ', rating_score=9.5), BusinessName(name='CogniTek', rating_score=8.8), BusinessName(name='Mindsync', rating_score=9.0), BusinessName(name='IntelliX', rating_score=9.3)]


Method 3 result:
names=[BusinessName(name='SynthAI', rating_score=9.2), BusinessName(name='NeuroIQ', rating_score=9.5), BusinessName(name='CogniTek', rating_score=8.8), BusinessName(name='Mindsync', rating_score=9.0), BusinessName(name='IntelliX', rating_score=8.7)]

JSON output:
{"names": [{"name": "SynthAI", "rating_score": 9.2}, {"name": "NeuroIQ", "rating_score": 9.5}, {"name": "CogniTek", "rating_score": 8.8}, {"name": "Mindsync", "rating_score": 9.0}, 